### Group Controller

- Load in a roster of Spheros
- Connect to each one and calibrate their position
- Perform Orchestration



In [1]:
# General Python Helper Functions
def readJsonFile(filename):
    '''
        Read a JSON file into a python dict 
    '''
    with open(filename) as data_file:    
        data = json.load(data_file)
    return data

In [2]:
# NEW SPHEROS
import sphero
import time

# Controller Functions
import os
import json
from pprint import pprint

In [3]:
# Use sphero app to get sphero names and MAC addresses
PROJ_ROOT = os.getcwd()
ROSTER = os.path.join(PROJ_ROOT, "roster.json")
roster = readJsonFile(ROSTER)
pprint(roster)

{u'68:86:E7:05:0C:4B': u'Sphero-YYP',
 u'68:86:E7:05:12:26': u'Sphero-RPB',
 u'68:86:E7:05:19:AD': u'Sphero-RYR',
 u'68:86:E7:09:4E:74': u'Sphero-YPR',
 u'68:86:E7:09:A2:FE': u'Sphero-RWR',
 u'68:86:E7:09:A6:FE': u'Sphero-ORG',
 u'68:86:E7:09:A9:28': u'Sphero-GRY'}


In [4]:
# Class for managing multiple spheros
manager = sphero.SpheroManager()

In [5]:
# Initialize Sphero manager with addresses of local spheros
manager._name_cache = roster

# Number of spheros to connect to
NBOTS = len(roster.keys())
print(NBOTS)

7


In [6]:
# This searches the system cache for all of the robots. 
# It doesn't need the spheros to be turned on yet.
def on_new_sphero(device, NBOTS=NBOTS):
    """
        NBOTS = number of robots in the flock
        Note that this presently refers to a global "manager" object. This method should be moved inside the native
        library later.
    """
    print "Found " + device.bt_name
    
    # Terminate search when all expected bots are found
    if len(manager._spheros.keys()) == NBOTS:
        print "Found all {} spheros".format(NBOTS)
        manager.stop_auto_search()
                 
# Callback
manager.set_sphero_found_cb(on_new_sphero)

# Construct list of devices in system bluetooth collection
# Your bluetooth network cannot contain anything with name of "Sphero-" prefix
devices = []
manager.start_auto_search()

Starts auto search
Found Sphero-RPB
Found Sphero-GRY
Found Sphero-YYP
Found Sphero-ORG
Found Sphero-RYR
Found Sphero-RWR
Found Sphero-YPR
Found all 7 spheros


In [7]:
devices = []
# Verify that these are the robots we are looking for
for name, device in manager._spheros.iteritems():
    print "{}: {}".format(name, device.bt_addr)
    devices.append(device) 

Sphero-RPB: 68:86:E7:05:12:26
Sphero-YPR: 68:86:E7:09:4E:74
Sphero-GRY: 68:86:E7:09:A9:28
Sphero-YYP: 68:86:E7:05:0C:4B
Sphero-RWR: 68:86:E7:09:A2:FE
Sphero-ORG: 68:86:E7:09:A6:FE
Sphero-RYR: 68:86:E7:05:19:AD


In [8]:
activeBotNames = [
    "Sphero-RYR",
    "Sphero-GRY",
    "Sphero-YPR",
    "Sphero-RWR",
    "Sphero-ORG",
    "Sphero-RPB",
    "Sphero-YYP"
]

These are experiments on 4/15


In [147]:
# Team control methods
    
def connect_team(bots):
    for i, bot in enumerate(bots):  
        bot.disconnect()
        bot.connect()
        
def disconnect_team(bots):
    for i, bot in enumerate(bots):  
        bot.disconnect()
        
def set_team_back_led(bots, status):
    # Bright if true, dim if false
    status = 0xaa if status else 0x00
    
    for bot in bots:  
        bot.set_back_led_output(status)
        
def set_team_colors(bots, colors):
    for i, bot in enumerate(bots):  
        colorTriple = colors[i]
        bot.set_rgb(colorTriple[0], colorTriple[1], colorTriple[2])
    
# Diagnostics

def highlight_bot(bots, iBot):
    for i, bot in enumerate(bots):
        if i == iBot:
            bot.set_rgb(255, 0, 0)
        else:
            bot.set_rgb(0, 0, 0)
            
def highlight_team(bots, duration=1):
    for i, bot in enumerate(bots):
        highlight_bot(bots, i)
        time.sleep(duration)
    
def print_team_status(bots):
    for bot in bots:
        response = bot.get_power_state()
        print "{} {} | {}".format(bot.bt_name, response.power_state, response.bat_voltage)
      
    return None
    
# MOVEMENT   
def roll_sphero(bot, speed, heading, offset):
    """
        Roll robot in in proper direction at a given speed
    """
    bot.roll(speed, normalize_angle(heading + offset))
    
def set_team_timeout(bots, motionTimeout=2000):
    """
        Move all robots in same direction at shared speed
    """
    for bot in bots:
        bot.set_motion_timeout(motionTimeout)

        
def roll_sphero_team_synchronized(bots, speed, heading, offsets,motionTimeout=2000):
    """
        Move all robots in same direction at shared speed
    """
#     for bot in bots:
#         bot.set_motion_timeout(motionTimeout)
    assert(len(bots) == len(offsets))
    tStart = time.time()
    
    for i, bot in enumerate(bots):
        roll_sphero(bot, speed, heading, offsets[i])
        
    tEnd = time.time()
    
    print("Dispatch Time {}".format(tEnd - tStart) )
    time.sleep(motionTimeout / 1000)  # wait for bots to finish rolling
    
# Math Functions
def normalize_angle(angle):
    if angle < 0:
        return 360 + angle
    elif angle > 359:
        return angle - 360
    else:
        return angle

First, initialize robot objects


In [10]:
# Test notes 4/22
bots = [ manager._spheros[name] for name in activeBotNames ]

In [169]:
# Connect team to computer
connect_team(bots)

Sphero-RYR try 0
An exception of type IOError occurred. Arguments:
('A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n',)
Sphero-RYR try 1
Connected!
Sphero-GRY try 0
Connected!
Sphero-YPR try 0
Connected!
Sphero-RWR try 0
An exception of type IOError occurred. Arguments:
('A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n',)
Sphero-RWR try 1
Connected!
Sphero-ORG try 0
An exception of type IOError occurred. Arguments:
('A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond.\r\n',)
Sphero-ORG try 1
Connected!
Sphero-RPB try 0
Connected!
Sphero-YYP try 0
An exception of type IOError occurr

In [170]:
# Make sure everyone's battery is above 7 volts. Below that they turn off.
# Get roughly 2-3 hours of ontime (not continuous)
print_team_status(bots)

Sphero-RYR Battery OK | 7.31
Sphero-GRY Battery OK | 7.33
Sphero-YPR Battery OK | 7.27
Sphero-RWR Battery OK | 7.3
NOTE! Removes wrong byte in start of header! Byte:  117
NOTE! Removes wrong byte in start of header! Byte:  62
Sphero-ORG Battery OK | 7.3
NOTE! Removes wrong byte in start of header! Byte:  117
NOTE! Removes wrong byte in start of header! Byte:  62
Sphero-RPB Battery OK | 7.38
NOTE! Removes wrong byte in start of header! Byte:  117
NOTE! Removes wrong byte in start of header! Byte:  62
Sphero-YYP Battery OK | 7.3


In [59]:
# Manage sphero colors consistently
colors = [
    [255, 0 , 0], # R
    [0, 255 , 0], # G
    [0, 0 , 255], # B
    
    [255, 0, 255],   # Purple
    [255, 255, 0],   # Yellow
    [255, 133, 0],   # Orange
    [255, 192, 203], # pink
]

# Colors to cut down
offCols = [ [0, 0, 0] for bot in bots]

POWER_SAVE = False

# Control Brightness
if POWER_SAVE:
    for color in colors:
        for i, val in enumerate(color):
            color[i] /= 2

# Initialization for any team
set_team_back_led(bots, True)


In [148]:
# illuminate robots from 1 to 7, one at a time so we know which one will receive commands first
highlight_team(bots, 1) # keyword = duration to wait in between pulses

In [172]:
# Turn colors off to save power
set_team_colors(bots, offCols)

Now, we must calibrate each robot's sense of direction


In [60]:
# 
highlight_bot(bots, 6)

In [127]:
bots[4].roll(70, )

In [173]:
# This process needs to be tuned by hand each time the robots fall asleep / get turned on!
# It would be nice to calibrate with the camera's help, but it's also ok if there isn't time.
# Eventually these offsets could be moved to properties of the sphero object.
# Recommend setting the 0 direction to be parallel to one of the walls.

botAngleOffsets = [
    115, 
    115, 
    95, 
    112, 
    125, 
    73, 
    118   
]

nullOffsets = [0 for bot in bots]
set_team_colors(bots, colors)

In [129]:
# for i,offset in enumerate(botAngleOffsets):
#     bots[i].set_heading(offset)
    

In [130]:
for i in range(7):
    time.sleep(1)
    highlight_bot(bots, i)
    

In [167]:
# Demo Reel: show basic polygon

corners = 4
angles = [ i * (360 / corners) for i in range(corners)]

# angles = [180]

TIMEOUT = 1500
set_team_timeout(bots, TIMEOUT)
for angle in angles:
    roll_sphero_team_synchronized(bots, 80, angle, botAngleOffsets, TIMEOUT)   

Dispatch Time 0.438999891281


In [ ]:
# Demo Reel: Laps Back and Forth
laps = 2
corners = 2
angles = [ i * (360 / corners) for i in range(corners)]

# angles = [180]

TIMEOUT = 4000
set_team_timeout(bots, TIMEOUT)
for lap in range(laps):
    for angle in angles:
        roll_sphero_team_synchronized(bots, 100, angle, botAngleOffsets, TIMEOUT)   

In [ ]:
# Demo Reel: Basic Push
# Strategy: rewind and repush each time?

angles = [0]
TIMEOUT = 4000 # milliseconds of pushing
set_team_timeout(bots, TIMEOUT)
for angle in angles:
    roll_sphero_team_synchronized(bots, 80, angle, botAngleOffsets, TIMEOUT)   

In [ ]:
# Demo Reel: Advanced Push
# Strategy: rewind and repush each time?

ANGLES =   [0, 180]
TIMEOUTS = [3000, 2000] # milliseconds of pushing

for i in range(3): # number of cycles to repush
for timeout in TIMEOUTS:
    
    set_team_timeout(bots, timeout)
    for angle in angles:
        roll_sphero_team_synchronized(bots, 80, angle, botAngleOffsets, TIMEOUT)   